In [ ]:
library(data.table)
library(gganimate)
library(cowplot)

In [ ]:
# create animation; based on example at https://github.com/nfl-football-ops/Big-Data-Bowl 
# see the NFL Data Preparation kernel for source code producing the csv file
example.play = fread('../input/nfl-data-preparation/animate_585_733.csv')
example.play[Team == 'ball', Number := ":"]
example.play[, Number := substr(Number, 1, 3)]
example.play[Number == 55, involved := 1]
example.play[Team == 'ball', involved := 2]
example.play[, involved := as.character(involved)]
setorder(example.play, -Team)


# set field params
ymin <- 0
ymax <- 160/3
xmin <- 0
xmax <- 120

hash.top <- 38.35
hash.bottom <- 12
hash.width <- 3.3
df.hash <- expand.grid(y = c(0, 18, ymax-18, ymax), x = (10:110))


# create base plot
animate.play <- ggplot() +
  
  geom_rect(data=df.hash, xmin=xmin, xmax=xmax, ymin=ymin, ymax=ymax, fill="#88c999", alpha=0.2) +
  
    annotate("segment", x = c(xmin, xmin, xmax, xmax), 
           y = c(ymin, ymax, ymax, ymin), 
           xend = c(xmin, xmax, xmax, xmin), 
           yend = c(ymax, ymax, ymin, ymin), colour = "darkgreen") + 
  
    annotate("segment", y = ymin, 
           x = seq(max(10, xmin), min(xmax, 110), by = 5), 
           yend =  ymax, 
           xend = seq(max(10, xmin), min(xmax, 110), by = 5), color="white") + 
  
    annotate("text", x = df.hash$x[df.hash$y < 55/2],
           y = df.hash$y[df.hash$y < 55/2], label = "_", hjust = 0, vjust = -0.2, 
          angle=90, color="white") +
 
    annotate("text", x = df.hash$x[df.hash$y > 55/2],
           y = df.hash$y[df.hash$y > 55/2], label = "_", hjust = 1, vjust = -0.2, 
           angle=90, color="white") +
  
    annotate("text", y = rep(hash.bottom - 2, 11), x = seq(10, 110, by = 10),
           label = c(" G", seq(10, 50, by = 10), rev(seq(10, 40, by = 10)), "G "),
           angle = 0, size = 4, color="white") +
    
    annotate("text", y = rep((ymax - hash.bottom + 3), 11), x = seq(10, 110, by = 10), 
           label = c("G  ", seq(10, 50, by = 10), rev(seq(10, 40, by = 10)), " G"), 
           angle = 180, size = 4, color="white") + 
  
  geom_point(data = example.play, aes(x = x, y = y, 
                    colour = Team, group = GSISID, pch = involved, size = involved)) + 
  
  scale_size_manual(values = c(7, 7, 4), guide = FALSE) + 
  scale_shape_manual(values = c(19, 17, 19), guide = FALSE) +
  scale_colour_manual(values = c("#654321", "#002244", "#e31837"), guide = FALSE) + 

  geom_text(data = example.play, aes(x = x, y = y, label = Number), colour = "white", 
            hjust = 0.4, size = 3.5) + 

  ylim(ymin, ymax) + 
  coord_fixed() +  
  theme_nothing() + 
  transition_time(frame_id)  +
  ease_aes('linear') + 
  NULL

In [ ]:
# animate
play.length.ex <- length(unique(example.play$frame_id))
anim = animate(animate.play, fps = 10, nframe = play.length.ex)
anim_save("animation_585_733.gif", anim)